In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sqlalchemy import create_engine
import warnings
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler


warnings.filterwarnings('ignore')

postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'useducation'

engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))

useducation_df = pd.read_sql_query('select * from useducation',con=engine)

engine.dispose()

In [18]:
fill_list = ["ENROLL", "TOTAL_REVENUE", "FEDERAL_REVENUE",
             "STATE_REVENUE", "LOCAL_REVENUE", "TOTAL_EXPENDITURE",
             "INSTRUCTION_EXPENDITURE", "SUPPORT_SERVICES_EXPENDITURE",
             "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE", 
             "GRADES_PK_G","GRADES_KG_G", "GRADES_4_G", "GRADES_8_G", 
             "GRADES_12_G", "GRADES_1_8_G","GRADES_9_12_G", "GRADES_ALL_G"]

for state in useducation_df["STATE"].unique():
    useducation_df.loc[useducation_df["STATE"] == state, fill_list] = useducation_df.loc[useducation_df["STATE"] == state, fill_list].interpolate()

# we drop the null values after interpolation
useducation_df.dropna(inplace=True)



1. Create a new score variable from the weighted averages of all score variables in the datasets. Notice that the number of students in the 4th grade isn't the same as the number of students in the 8th grade. So, you should appropriately weigh the scores!

In [19]:
weight_factor = 1 / (useducation_df["GRADES_4_G"] + useducation_df["GRADES_8_G"])
g_8_unweighted = useducation_df["GRADES_8_G"] * ((useducation_df["AVG_MATH_8_SCORE"] + useducation_df["AVG_READING_8_SCORE"])*0.5)
g_4_unweighted = useducation_df["GRADES_4_G"] * ((useducation_df["AVG_MATH_4_SCORE"] + useducation_df["AVG_READING_4_SCORE"])*0.5)

useducation_df["weighted_average"] = (g_4_unweighted + g_8_unweighted) * weight_factor




2. What are the correlations between this newly created score variable and the expenditure types? Which 1 of the expenditure types is more correlated than the others?

In [20]:
useducation_df[["weighted_average", "TOTAL_EXPENDITURE", 
                "INSTRUCTION_EXPENDITURE","SUPPORT_SERVICES_EXPENDITURE", 
                "OTHER_EXPENDITURE", "CAPITAL_OUTLAY_EXPENDITURE"]].corr()

,weighted_average,TOTAL_EXPENDITURE,INSTRUCTION_EXPENDITURE,SUPPORT_SERVICES_EXPENDITURE,OTHER_EXPENDITURE,CAPITAL_OUTLAY_EXPENDITURE
weighted_average,1.000000,0.074155,0.087386,0.079002,0.010959,-0.005389
TOTAL_EXPENDITURE,0.074155,1.000000,0.990255,0.992988,0.943702,0.925784
INSTRUCTION_EXPENDITURE,0.087386,0.990255,1.000000,0.974399,0.902173,0.884798
SUPPORT_SERVICES_EXPENDITURE,0.079002,0.992988,0.974399,1.000000,0.952282,0.911245
OTHER_EXPENDITURE,0.010959,0.943702,0.902173,0.952282,1.000000,0.918188
CAPITAL_OUTLAY_EXPENDITURE,-0.005389,0.925784,0.884798,0.911245,0.918188,1.000000


The most closely correlated value is instruction expenditure (r = 0.087)

3. Now, apply PCA to the 4 expenditure types. How much of the total variance is explained by the 1st component?

In [26]:
sklearn_pca = PCA(n_components=1)

X = useducation_df[["INSTRUCTION_EXPENDITURE", 
                  "SUPPORT_SERVICES_EXPENDITURE",
                  "OTHER_EXPENDITURE", 
                  "CAPITAL_OUTLAY_EXPENDITURE"]]

X = StandardScaler().fit_transform(X)
useducation_df["pca_1"] = sklearn_pca.fit_transform(X)

print(
    'The percentage of total variance in the dataset explained by each',
    'component from Sklearn PCA.\n',
    sklearn_pca.explained_variance_ratio_
)

The percentage of total variance in the dataset explained by each component from Sklearn PCA.
 [0.9430175]


4. What is the correlation between the overall score variable and the 1st principal component?

In [28]:
useducation_df[["weighted_average", "pca_1"]].corr()

,weighted_average,pca_1
weighted_average,1.000000,0.044596
pca_1,0.044596,1.000000


5. If you were to choose the best variables for your model, would you prefer using the 1st principal component instead of the expenditure variables? Why?

The first principal component only explains 4.4% of the variance in the weigted average reading and math scores, whereas expenditure on instruction explains 8.7%. Neither are very strong, but instruciton expenditure is the best choice among variables to use in a model explaining variance in test scores.  